In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ['LANGCHAIN_TRACING_V2'] = "true"


## Loading the data

In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader('https://en.wikipedia.org/wiki/Alfred_Nobel')
document = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


## Splitting the document in chunks

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size = 2000,chunk_overlap = 400)

splitted_document = splitter.split_documents(document)
print(splitted_document)
##print(splitted_document[1])

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Alfred_Nobel', 'title': 'Alfred Nobel - Wikipedia', 'language': 'en'}, page_content='Alfred Nobel - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nBiog

In [4]:
from langchain_ollama import OllamaEmbeddings

embedding = OllamaEmbeddings(model = 'embeddinggemma:300m')

In [5]:
from langchain_community.vectorstores import FAISS

vector_db = FAISS.from_documents(splitted_document,embedding)

## Quering from vector db

In [6]:
query = 'what did alfred nodel invented ?'
response = vector_db.similarity_search(query)
response

[Document(id='2ada3e67-4250-4cf7-9524-c7283c48239b', metadata={'source': 'https://en.wikipedia.org/wiki/Alfred_Nobel', 'title': 'Alfred Nobel - Wikipedia', 'language': 'en'}, page_content='Toggle Personal life subsection\n\n\n\n\n\n2.1\nReligion\n\n\n\n\n\n\n\n\n2.2\nRomantic relationships and personality\n\n\n\n\n\n\n\n\n2.3\nResidences\n\n\n\n\n\n\n\n\n\n\n3\nMonument to Alfred Nobel\n\n\n\n\n\n\n\n\n4\nCriticism\n\n\n\n\nToggle Criticism subsection\n\n\n\n\n\n4.1\nAntisemitism\n\n\n\n\n\n\n\n\n\n\n5\nReferences\n\n\n\n\n\n\n\n\n6\nFurther reading\n\n\n\n\n\n\n\n\n7\nExternal links\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nToggle the table of contents\n\n\n\n\n\n\n\nAlfred Nobel\n\n\n\n161 languages\n\n\n\n\nAfrikaansAlemannischአማርኛÆngliscАԥсшәаالعربيةAragonésԱրեւմտահայերէնঅসমীয়াAsturianuAymar aruAzərbaycancaتۆرکجهBasa Baliবাংলা閩南語 / Bân-lâm-gíБашҡортсаБеларускаяБеларуская (тарашкевіца)भोजपुरीBikol CentralБългарскиBoarischབོད་ཡིགBosanskiBrezhonegCatalàЧӑвашлаCebuanoČeštinaCorsuCymraegDan

In [7]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model = 'gemma3')

## Retrival Chain and Document Chain

In [9]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
    Answer the provided question based on the following context:
    <context>
    {context}
    </context>
"""
)

document_chain = create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the provided question based on the following context:\n    <context>\n    {context}\n    </context>\n'), additional_kwargs={})])
| OllamaLLM(model='gemma3')
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

### Retrival Chain
above is the document chain , that will process the chat prompt by taking the input and answering accordingly , but to provide the context to that we will create a retrival chain , that will be getting the data from vector db and passing that to the document chain

In [10]:
from langchain.chains import create_retrieval_chain
retrival = vector_db.as_retriever()
retrival_chain = create_retrieval_chain(retrival,document_chain)
retrival_chain


RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x11080c050>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the provided question based on the following context:\n    <context>\n    {context}\n    </context>\n'), additional_kwargs={})])
            | 

## Now we will invoke this retriva chain 

In [11]:
response =retrival_chain.invoke({"input":"Who was Alfred_Nobel and what did he invent"})
response['answer']

'Based on the provided context, here’s a summary of Alfred Nobel’s life and legacy:\n\n*   **Inventor and Businessman:** Alfred Nobel was a Swedish chemist and engineer best known for inventing dynamite. He founded Dynamit Nobel, a company that produced explosives.\n*   **Creation of the Nobel Prizes:** After his death, Nobel established the Nobel Prizes – Physics, Chemistry, Physiology or Medicine, Literature, and Peace – to honor those who "conferred the greatest benefit to humankind."\n*   **Royal Swedish Academy of Sciences:** He was a member of the Royal Swedish Academy of Sciences, which continues to select the Nobel laureates in Physics and Chemistry according to his will.\n*   **Company Connections:** Several companies, including Dynamit Nobel and AkzoNobel, descend from his original companies.\n*   **Premature Death:** Nobel died prematurely from angina (chest pain) likely due to his extensive exposure to nitroglycerine, a key ingredient in dynamite.\n*   **Controversial Legac

In [12]:
response['context']

[Document(id='0b8e91d2-18c0-4296-9a14-2adfa5f94ecc', metadata={'source': 'https://en.wikipedia.org/wiki/Alfred_Nobel', 'title': 'Alfred Nobel - Wikipedia', 'language': 'en'}, page_content='Upon his death, Nobel donated his fortune to a foundation to fund the Nobel Prizes, which annually recognize those who "conferred the greatest benefit to humankind".[3][4] The synthetic element nobelium was named after him,[5] and his name and legacy also survive in companies such as Dynamit Nobel and AkzoNobel, which descend from mergers with companies he founded. Nobel was elected a member of the Royal Swedish Academy of Sciences, which, pursuant to his will, is responsible for choosing the Nobel laureates in Physics and in Chemistry.'),
 Document(id='92cd278a-2684-49c5-a8e4-5658f205e25f', metadata={'source': 'https://en.wikipedia.org/wiki/Alfred_Nobel', 'title': 'Alfred Nobel - Wikipedia', 'language': 'en'}, page_content='^ "Alfred Nobel – Dynamit" [Alfred Nobel – Dynamite] (in Swedish). Swedish N